<a href="https://colab.research.google.com/github/The-DigitalAcademy/TrendHunt-Application/blob/Model/final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential  # Import the Sequential class
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [2]:

# Define data directories
train_data_dir = '/content/drive/MyDrive/my data/train'
validation_data_dir = '/content/drive/MyDrive/my data/validation'
test_data_dir = '/content/drive/MyDrive/my data/test'

In [4]:
# Define data preprocessing parameters
batch_size = 32
input_shape = (150, 150)  # Adjust to your image size

In [5]:
# Data Augmentation for training data (optional)
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [6]:
# Load and preprocess the data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'  # This will generate one-hot encoded labels
)

Found 69 images belonging to 6 classes.


In [7]:

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 70 images belonging to 6 classes.


In [8]:
# Load and preprocess test data (without data augmentation)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 0 images belonging to 0 classes.


In [12]:

# Define the number of classes
num_classes = len(train_generator.class_indices)
num_classes

6

Model building

In [16]:
# Create a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])



In [17]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [20]:
# Train the model
epochs = 20
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/20
2/2 [==============================] - 5s 4s/step - loss: 1.6918 - accuracy: 0.3514 - val_loss: 1.7375 - val_accuracy: 0.2500
Epoch 2/20
2/2 [==============================] - 4s 3s/step - loss: 1.6939 - accuracy: 0.2812 - val_loss: 1.7437 - val_accuracy: 0.2031
Epoch 3/20
2/2 [==============================] - 5s 4s/step - loss: 1.7181 - accuracy: 0.2432 - val_loss: 1.7174 - val_accuracy: 0.2188
Epoch 4/20
2/2 [==============================] - 3s 1s/step - loss: 1.6943 - accuracy: 0.2162 - val_loss: 1.7239 - val_accuracy: 0.2969
Epoch 5/20
2/2 [==============================] - 4s 3s/step - loss: 1.7153 - accuracy: 0.2031 - val_loss: 1.7158 - val_accuracy: 0.2656
Epoch 6/20
2/2 [==============================] - 6s 3s/step - loss: 1.7238 - accuracy: 0.2969 - val_loss: 1.6934 - val_accuracy: 0.2812
Epoch 7/20
2/2 [==============================] - 3s 3s/step - loss: 1.6826 - accuracy: 0.2432 - val_loss: 1.6904 - val_accuracy: 0.2344
Epoch 8/20
2/2 [=========================

In [21]:
# Extract feature vectors for all images in the dataset
feature_vectors = model.predict(train_generator)


3/3 [==============================] - 1s 221ms/step


In [23]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def preprocess_query_image(image_path):
    # Load the image
    img = load_img(image_path, target_size=(150, 150))
    # Convert the image to a NumPy array
    img_array = img_to_array(img)
    # Normalize the pixel values to the range [0, 1]
    img_array = img_array / 255.0
    return img_array


In [25]:
# Specify the path to your query image
query_image_path = '/content/drive/MyDrive/my data/test/fa064993-f459-457a-bdfb-69a2832e15da.jpg'

# Example query image (you should preprocess it the same way as the training images)
query_image = preprocess_query_image(query_image_path)

# Extract feature vector for the query image
query_features = model.predict(np.expand_dims(query_image, axis=0))  # Expand dimensions for a single image

# Calculate cosine similarity
similarities = cosine_similarity(query_features, feature_vectors)


1/1 [==============================] - 0s 97ms/step


testing

In [27]:
# Load sample images
sample_image_paths = ['/content/drive/MyDrive/my data/test/fa13626d-572c-428f-ac1c-c40e1673195b.jpg', '/content/drive/MyDrive/my data/test/fa633eb5-0ca4-4e22-943e-1cdeaf9e60b5.jpg']

# Preprocess sample images
sample_images = [preprocess_query_image(image_path) for image_path in sample_image_paths]


In [29]:
# Make predictions
sample_predictions = model.predict(np.array(sample_images))

# Convert class probabilities to class labels (if needed)
predicted_labels = np.argmax(sample_predictions, axis=1)
predicted_labels


1/1 [==============================] - 0s 41ms/step


array([3, 0])

In [30]:
# Define class labels (replace with your own class labels)
class_labels = ['Shirt', 'Dresses', 'Shoes','Skirts', 'T-shirt', 'Trouser']

# Display predictions for each sample image
for i in range(len(sample_image_paths)):
    print(f"Image: {sample_image_paths[i]}")
    print(f"Predicted Class: {class_labels[predicted_labels[i]]}")
    print(f"Class Probabilities: {sample_predictions[i]}")
    print()


Image: /content/drive/MyDrive/my data/test/fa13626d-572c-428f-ac1c-c40e1673195b.jpg
Predicted Class: Skirts
Class Probabilities: [0.12872005 0.05551829 0.17870551 0.30298096 0.08046499 0.2536102 ]

Image: /content/drive/MyDrive/my data/test/fa633eb5-0ca4-4e22-943e-1cdeaf9e60b5.jpg
Predicted Class: Shirt
Class Probabilities: [0.36460352 0.2061712  0.04295815 0.05211071 0.2686395  0.06551698]

